In [1]:
#!jupyter nbconvert --to script poc-migration-auto.ipynb

In [1]:
import datamigration_wrapper as dms
from datetime import datetime
import pandas as pd
import multiprocessing
from multiprocessing import log_to_stderr, get_logger
import logging
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--migration-file', type=str)
args = parser.parse_args("")

migration_file = args.migration_file or 'Test GCP Migration.csv'
df_machine_types = pd.read_csv('machine_types.csv')
df_migration_data = pd.read_csv(migration_file)

In [3]:
display(df_migration_data)

,Team Name,Service,k8sSecretNamespace,DB Identifier,Endpoint Address,Port,ReplicationUsername,ReplicationPassword,ProjectId,Version,Location,StorageType,CPU,Memory,Storage
0,accounts-identity,fake,tmc-iam,poc-migration-12,poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_12,us-east4,PD_SSD,2,1,15
1,accounts-identity,fake,tmc-iam,poc-migration-11,poc-migration-11.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_11,us-east4,PD_SSD,1,1,10
2,accounts-identity,fake,tmc-iam,poc-migration-12,poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_12,us-east4,PD_SSD,2,1,12
3,accounts-identity,fake,tmc-iam,poc-migration-11,poc-migration-11.cjq8ffbfbj4k.us-east-1.rds.am...,5432,postgres,postgres,aws-rds-gcp-cloudsql,POSTGRES_11,us-east4,PD_SSD,1,1,18


In [4]:
project_id = 'aws-rds-gcp-cloudsql'
region_id = 'us-east4'
prefix_cp_source = 'auto-cp-pg-'
prefix_cp_cloudsql = 'auto-cs-pg-'
prefix_mj='auto-mj-'
now_str = datetime.now().strftime("%Y%m%dt%H%M%S")

In [5]:
list_migration_job = []
for index, row in df_migration_data.iterrows():
    dc={}
    dc["source_connection"] = {"postgresql": {"host": row["Endpoint Address"],
                                      "port": row["Port"],
                                      "username": row["ReplicationUsername"],
                                      "password": row["ReplicationPassword"]}}
    dc["target_server_settings_cloud_sql"] = { "databaseVersion": row["Version"],
                              "tier": "db-custom-1-3840",
                              "dataDiskSizeGb": row["Storage"]}
    dc["location_dict"] = {"project_id": project_id,
                  "region_id": region_id}
    dc["prefix_dict"] = {"prefix_cp_source": prefix_cp_source,
               "prefix_cp_cloudsql": prefix_cp_cloudsql,
               "prefix_mj": prefix_mj,
               "now_str": now_str,
                "id" : index}
    dc["target_base_settings_cloud_sql"] = { "ipConfig": {
                                "enableIpv4": True
                              },
                              "autoStorageIncrease": True,
                              "dataDiskType": "PD_SSD",
                              "rootPassword": "postgres"}
    list_migration_job.append((dc["prefix_dict"],dc["location_dict"],dc["source_connection"],dc["target_base_settings_cloud_sql"],dc["target_server_settings_cloud_sql"]))

In [6]:
if __name__ == '__main__':
    
    jobs = []
    for (a,b,c,d,e) in list_migration_job:
        p = multiprocessing.Process(target=dms.generate_migration_job, args=(a,b,c,d,e,))
        p.start()
        jobs.append(p)
#for job in jobs:
#    job.join()

In [7]:
list_migration_job[0]

({'prefix_cp_source': 'auto-cp-pg-',
  'prefix_cp_cloudsql': 'auto-cs-pg-',
  'prefix_mj': 'auto-mj-',
  'now_str': '20210517t123551',
  'id': 0},
 {'project_id': 'aws-rds-gcp-cloudsql', 'region_id': 'us-east4'},
 {'postgresql': {'host': 'poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.amazonaws.com',
   'port': 5432,
   'username': 'postgres',
   'password': 'postgres'}},
 {'ipConfig': {'enableIpv4': True},
  'autoStorageIncrease': True,
  'dataDiskType': 'PD_SSD',
  'rootPassword': 'postgres'},
 {'databaseVersion': 'POSTGRES_12',
  'tier': 'db-custom-1-3840',
  'dataDiskSizeGb': 15})

In [8]:
val_list = []
for (a,b,c,d,e) in list_migration_job:
    val = dict()
    val["state"] = dms.get_state_from_migration_job(a,b,c)
    val["host"] = c["postgresql"]["host"]
    val_list.append(val)
pd.DataFrame(val_list)[["host","state"]]

,host,state
0,poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.am...,NOT FOUND
1,poc-migration-11.cjq8ffbfbj4k.us-east-1.rds.am...,NOT FOUND
2,poc-migration-12.cjq8ffbfbj4k.us-east-1.rds.am...,NOT FOUND
3,poc-migration-11.cjq8ffbfbj4k.us-east-1.rds.am...,NOT FOUND


In [12]:
def func(a,b,c,val):
    val["state"] = dms.get_state_from_migration_job(a,b,c)

In [23]:
manager = multiprocessing.Manager()
jobs = []
val_list = []
with multiprocessing.Manager() as manager:
    for (a,b,c,d,e) in list_migration_job:
        val = manager.dict()
        val_list.append(val)
        p = multiprocessing.Process(target= func, args=(a,b,c,val))
        p.start()
        jobs.append(p)
    for job in jobs:
        job.join()
    for idx, proc in enumerate(jobs):
        print('{}: {}'.format(idx, val_list[idx].values()))

0: []
1: []
2: []
3: []


In [20]:
print(val_list[0].__dict__)

{'_tls': <multiprocessing.util.ForkAwareLocal object at 0x000001B82F56D220>, '_idset': ProcessLocalSet({'201b16d5e00', '201b16d59c0', '201b16d5c80', '201b16d5980'}), '_token': Token(typeid='dict', address='\\\\.\\pipe\\pyc-19468-0-9kvxobde', id='201b16d59c0'), '_id': '201b16d59c0', '_manager': <multiprocessing.managers.SyncManager object at 0x000001B82F3983A0>, '_serializer': 'pickle', '_Client': <function Client at 0x000001B82F442AF0>, '_owned_by_manager': False, '_authkey': b'ik\x93=yzN\xa0\x98X+\x018\xce\x9f\xf7;\xc3o!5k\xb9a\xc1\xd6\xbcRp\x04O\x9e', '_close': <Finalize object, callback=_decref, args=(Token(typeid='dict', address='\\\\.\\pipe\\pyc-19468-0-9kvxobde', id='201b16d59c0'), b'ik\x93=yzN\xa0\x98X+\x018\xce\x9f\xf7;\xc3o!5k\xb9a\xc1\xd6\xbcRp\x04O\x9e', <multiprocessing.managers.State object at 0x000001B82F398460>, <multiprocessing.util.ForkAwareLocal object at 0x000001B82F56D220>, ProcessLocalSet({'201b16d5e00', '201b16d59c0', '201b16d5c80', '201b16d5980'}), <function Clie

NOT FOUND
